第二部分：PyTorch(进阶)

In [13]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

Point1:模型定义方式
PyTorch中自定义模型主要通过一下三种方式
- Sequential
- ModuleList
- ModuleDict

In [14]:
#讲解点：使用ordered dict更有助于一目了然模型结构，对于之后的模型修改也非常有帮助
##Sequential:Direct list(直接列出来)两种定义方式
import torch.nn as nn
net1=nn.Sequential(#做一个两层神经网络
       nn.Linear(784,256),#输入为784，隐层节点数256
       nn.ReLU(),#损失函数
       nn.Linear(256,10),
       )
print(net1)#两个Linear的映射层，一个Relu激活层

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [15]:
##Sequential:Ordered Dict#带顺序的字典
import collections
import torch.nn as nn
net2=nn.Sequential(collections.OrderedDict([
       ('fcl',nn.Linear(784,256)),
       ('relul',nn.ReLU()),
       ('fc2',nn.Linear(256,10))
       ]))
print(net2)
#两个一样的
#实现层名称的命名，比较方便实现对应功能

Sequential(
  (fcl): Linear(in_features=784, out_features=256, bias=True)
  (relul): ReLU()
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [16]:
#试一下
a=torch.rand(4,784)#随机产生一个tensor
out1=net1(a)#输出结果
out2=net2(a)
print(out1.shape==out2.shape,out1.shape)

True torch.Size([4, 10])


In [17]:
#ModuleList
#层比较多，且一样
net3=nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
net3.append(nn.Linear(256,10))#类似list的append操作
print(net3[-1])
print(net3)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [18]:
#注意ModuleList 并没有定义一个网络，他只是将不同的模块储存在一起，此处此处应报错
#out3=net3(a)

In [19]:
class Net3(nn.Module):
    def __init__(self):#先初始化
        super().__init__()#利用初始化定义模型层
        self.modulelist=nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
        self.modulelist.append(nn.Linear(256,10))
    def forward(self,x):#得到前向传播的关系
        for layer in self.modulelist:
            x=layer(x)
        return x
net3_ = Net3()
out3_ = net3_(a)
print(out3_.shape)#得到结果

torch.Size([4, 10])


In [20]:
#ModuleDict
net4=nn.ModuleDict({#做一个两层神经网络
       'linear':nn.Linear(784,256),#输入为784
       'act':nn.ReLU(),#损失函数
})
net4['output']=nn.Linear(256,10)#添加第三层
print(net4['linear'])
print(net4.output)#两个Linear的映射层，一个Relu激活层

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)


In [119]:
#同样的，ModuleDict并没有定义一个网络，他只是将不同的模块储存在一起，此处应报错
#正确使用的方法同上
#out4=net(a)

## 利用模块快速搭建复炸网路
      快速构建U-Net
      定义好几个模块，然后串起来，模块可以复用，U-Net每一个小块都有两次卷积的操作

组成U-Net的模型主要包括：
- 每个子块内部的两次卷积（Double Convolution)
- 左侧模型块之间的下采样链接，及最大池化（Max pooling）
- 右侧模型块之间的上采样连接（Up sampling）
- 输出层的处理
      除模型块外，还有模型块之间的，横向链接，输入和U-Net底部的链接计算，通过forward实现

In [56]:
import os
import numpy as np
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [57]:
class DoubleConv(nn.Module):#双次卷积
    '''(convolution =>[BN]>=ReLU) *2'''
    def __init__(self,in_channels,out_channels,mid_channels=None):#三个通道
        super().__init__()
        if not mid_channels:
            mid_channels=out_channels
        self.double_conv=nn.Sequential(
            nn.Conv2d(in_channels,mid_channels,kernel_size=3,padding=1,bias=False),#第一次卷积
            nn.BatchNorm1d(mid_channels),#忘了这是干啥来着
            nn.ReLU(inplace=True),#激活
            nn.Conv2d(mid_channels,out_channels,kernel_size=3,padding=1,bias=False),#第二次卷积
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self,x):
        return self.double_conv(x)

In [58]:
class Down(nn.Module):
    '''Downscaling with maxpool then double conv'''
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.maxpool_conv=nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels,out_channels)#复用
        )
    def forward(self,x):#得到输出
        return self.maxpool_conv(x)

In [59]:
class Up(nn.Module):
    '''UPscaling then don=uble conv'''
    def __init__ (self,in_channels,out_channels,bilinear=True):
        super().__init__()
     #if bilinear,use the normal convolutions to reduce number of channels
        if bilinear: #'Up sampling是上采样过程，bilinear是差值 内置的的函数
            self.up=nn.Upsample(scale_factor=2,mode='bilinear',align_corners=True)#上采样操作
            self.conv=DoubleConv(in_channels,out_channels,in_channels//2)#卷积
        else:
            self.up=nn.ConvTranspose2d(in_channels,in_channels//2,kernel_size=2,stride=2)
            self.conv=DoubleConv(in_channels,out_channels)
        
        def forward(self,x1,x2):
            x1=self.up(x1)
        #input is CHW
            diffY =x2.size()[2]-x1.size()[2]
            diffX =x2.size()[3]-x1.size()[3]
        
            x1=F.pad(x1,[diffX//2,diffX - diffX//2,
                    diffY//2,diffY-diffY//2])
        #if you have padding issues,see
        #https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/
        #https://github.com/xiaopeng-liao/Pytorch-UNet
        x=torch.cat([x2,x1],dim=1)#重要的将x1与x2放在一起  数据流和数据
        return self.conv(x)

In [99]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [100]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

In [101]:
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

In [102]:
class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

In [103]:
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [104]:
#组装
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):#链接
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [105]:
unet=UNet(3,1)#输入的是三个channel,输出维度为1
unet
#确实感觉很神奇，输出的结果是啥意思呢？

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

Pint3:模型修改
这里假设最后的翻个是多类别的（即mask不止0和1，还有2，3，4等值代表其他目标），需要对模型特定层经行修改。
此外还有两种情况的模型修改方案
- 添加额外输入
- 添加额外输出

In [106]:
#修改特定层
import torchvision.models as models
net = models.resnet50()
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [107]:
##这里模型结构是为了适配ImageNet预训练的权重，因此最后全连接层（fc）的输出节点数是1000。

In [108]:
#假设我们要用这个resnet模型去做一个10分类的问题，就应该修改模型的fc层，将其输出节点数替换为10。另外，我们觉得一层全连接层可能太少了，想再加一层。可以做如下修改：
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(2048, 128)),
                          ('relu1', nn.ReLU()), 
                          ('dropout1',nn.Dropout(0.5)),
                          ('fc2', nn.Linear(128, 10)),
                          ('output', nn.Softmax(dim=1))
                          ]))
    
net.fc = classifier
#这里的操作相当于将模型（net）最后名称为“fc”的层替换成了名称为“classifier”的结构，该结构是我们自己定义的。这里使用了第一节介绍的Sequential+OrderedDict的模型定义方式。

- 添加外部输入

In [109]:
?nn.Conv2d

In [110]:
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [111]:
#我们以torchvision的resnet50模型为基础，任务还是10分类任务。不同点在于，我们希望利用已有的模型结构，在倒数第二层增加一个额外的输入变量add_variable来辅助预测。具体实现如下：
class Model(nn.Module):
    def __init__(self, net):
        super(Model, self).__init__()
        self.net = net
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc_add = nn.Linear(1001, 10, bias=True)#改变
        self.output = nn.Softmax(dim=1)
        
    def forward(self, x, add_variable):
        x = self.net(x)
        x = torch.cat((self.dropout(self.relu(x)), add_variable.unsqueeze(1)),1)
        x = self.fc_add(x)
        x = self.output(x)
        return x

In [112]:
#对我们修改好的模型结构进行实例化
import torchvision.models as models
net = models.resnet50()
#model = Model(net).cuda()
#model = Model(net)

In [114]:
#训练中在输入数据的时候要给两个inputs
outputs = model(inputs,add_var)

NameError: name 'inputs' is not defined

- 添加额外输出

In [115]:
class Model(nn.Module):
    def __init__(self, net):
        super(Model, self).__init__()
        self.net = net
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(1000, 10, bias=True)
        self.output = nn.Softmax(dim=1)
        
    def forward(self, x, add_variable):
        x1000 = self.net(x)
        x10 = self.dropout(self.relu(x1000))
        x10 = self.fc1(x10)
        x10 = self.output(x10)
        return x10, x1000

In [117]:
import torchvision.models as models
net = models.resnet50()
#model = Model(net).cuda()

In [118]:
out10, out1000 = model(inputs, add_var)

NameError: name 'inputs' is not defined

-------------------*********-------------------------------------

Pint3:模型修改 这里假设最后的翻个是多类别的（即mask不止0和1，还有2，3，4等值代表其他目标），需要对模型特定层经行修改。 此外还有两种情况的模型修改方案

- 添加额外输入
- 添加额外输出

In [130]:
## 修改特定层
import copy
unet1=copy.deepcopy(unet)#复制UENT结构
unet1.outc#这里是检验outcome层的样子

OutConv(
  (conv): Conv2d(64, 1, kernel_size=(1, 1), stride=(1, 1))
)

In [137]:
b=torch.rand(1,3,224,224)#1是bithsize,3是通道数，224是大小
out_unet1=unet(b)
print(out_unet1.shape)

torch.Size([1, 1, 224, 224])


In [138]:
#unet1

In [139]:
#?torch.rand

In [140]:
#?torch.Size

In [141]:
unet1.outc=OutConv(64,5)
unet1.outc

OutConv(
  (conv): Conv2d(64, 5, kernel_size=(1, 1), stride=(1, 1))
)

In [142]:
out_unet=unet1(b)
print(out_unet1.shape)
#为啥我不变？

torch.Size([1, 1, 224, 224])


In [143]:
#添加额外输入
class UNet2(nn.Module):
    def __init__(self,n_channels,n_classes,bilinear=True):
        super(UNet2,self).__init__()
        self.n_channels=n_channels
        self.n_classes = n_classes
        self.bilinear=bilinear
        
        self.inc=DoubleConv(n_channels,64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)
         
    def forward(self, x,add_variable):#增加add_variable
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x=x+add_variable  #修改点
        logits = self.outc(x)
        return logits
unet2=UNet2(3,1)

c=torch.rand(1,1,224,224)#干扰项
out_unet2=unet2(b,c)
print(out_unet2.shape)

torch.Size([1, 1, 224, 224])


In [148]:
#添加输出
class UNet3(nn.Module):
    def __init__(self,n_channels,n_classes,bilinear=True):
        super(UNet3,self).__init__()
        self.n_channels=n_channels
        self.n_classes = n_classes
        self.bilinear=bilinear
        
        self.inc=DoubleConv(n_channels,64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)
         
    def forward(self, x):#增加add_variable
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)#最下层，修改，将其return出来
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        
        logits = self.outc(x)
        return logits,x5  #修改点
unet3 = UNet3(3,1)

c=torch.rand(1,1,224,224)#干扰项
out_unet3,mid_out=unet3(b)#增加输出，接住return出来的x5
print(out_unet3.shape,mid_out.shape)

torch.Size([1, 1, 224, 224]) torch.Size([1, 512, 14, 14])


print4:模型保存与读取
  这里考虑单卡和多卡情况下的模型存取情况

In [149]:
## 回到jupyter notebook. 看保存结果

In [150]:
unet

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [151]:
unet.state_dict()

OrderedDict([('inc.double_conv.0.weight',
              tensor([[[[-1.0233e-01, -6.8659e-02,  1.4602e-01],
                        [-1.4983e-02,  1.0415e-01,  4.8546e-02],
                        [-1.8698e-01,  1.9076e-01,  6.8710e-02]],
              
                       [[ 4.5962e-02, -6.6551e-02, -1.8086e-02],
                        [ 4.1419e-03,  1.7666e-02,  1.0930e-01],
                        [-1.2438e-01, -9.1674e-02,  1.7823e-01]],
              
                       [[-1.0608e-02, -9.8779e-02, -1.3818e-01],
                        [-6.7015e-02, -7.4159e-02,  3.5327e-02],
                        [ 1.6893e-01, -5.1772e-02, -1.7725e-01]]],
              
              
                      [[[-1.8209e-01, -1.7253e-01, -1.6814e-01],
                        [ 1.2004e-01,  6.3147e-02, -5.4056e-02],
                        [-9.4213e-02, -9.3810e-03, -1.0236e-01]],
              
                       [[ 1.3078e-01,  1.1881e-02,  1.1509e-01],
                        [-1.6733e

In [152]:
##cpu或单卡：保存&读取整个模型
torch.save(unet,'./unet_example.pth')
loaded_unet = torch.load('./unet_example.pth')
loaded_unet.state_dict()

OrderedDict([('inc.double_conv.0.weight',
              tensor([[[[-1.0233e-01, -6.8659e-02,  1.4602e-01],
                        [-1.4983e-02,  1.0415e-01,  4.8546e-02],
                        [-1.8698e-01,  1.9076e-01,  6.8710e-02]],
              
                       [[ 4.5962e-02, -6.6551e-02, -1.8086e-02],
                        [ 4.1419e-03,  1.7666e-02,  1.0930e-01],
                        [-1.2438e-01, -9.1674e-02,  1.7823e-01]],
              
                       [[-1.0608e-02, -9.8779e-02, -1.3818e-01],
                        [-6.7015e-02, -7.4159e-02,  3.5327e-02],
                        [ 1.6893e-01, -5.1772e-02, -1.7725e-01]]],
              
              
                      [[[-1.8209e-01, -1.7253e-01, -1.6814e-01],
                        [ 1.2004e-01,  6.3147e-02, -5.4056e-02],
                        [-9.4213e-02, -9.3810e-03, -1.0236e-01]],
              
                       [[ 1.3078e-01,  1.1881e-02,  1.1509e-01],
                        [-1.6733e

In [154]:
##cpu或单卡：保存&读取整个权重
torch.save(unet.state_dict(),'./unet_example.pth')
loaded_unet_weights = torch.load('./unet_example.pth')
loaded_unet.state_dict(loaded_unet_weights)
unet.state_dict()

OrderedDict([('inc.double_conv.0.weight',
              tensor([[[[-1.0233e-01, -6.8659e-02,  1.4602e-01],
                        [-1.4983e-02,  1.0415e-01,  4.8546e-02],
                        [-1.8698e-01,  1.9076e-01,  6.8710e-02]],
              
                       [[ 4.5962e-02, -6.6551e-02, -1.8086e-02],
                        [ 4.1419e-03,  1.7666e-02,  1.0930e-01],
                        [-1.2438e-01, -9.1674e-02,  1.7823e-01]],
              
                       [[-1.0608e-02, -9.8779e-02, -1.3818e-01],
                        [-6.7015e-02, -7.4159e-02,  3.5327e-02],
                        [ 1.6893e-01, -5.1772e-02, -1.7725e-01]]],
              
              
                      [[[-1.8209e-01, -1.7253e-01, -1.6814e-01],
                        [ 1.2004e-01,  6.3147e-02, -5.4056e-02],
                        [-9.4213e-02, -9.3810e-03, -1.0236e-01]],
              
                       [[ 1.3078e-01,  1.1881e-02,  1.1509e-01],
                        [-1.6733e

In [160]:
## 多卡：保存&读取整个模型。注意模型层名称前多了module
## 不建议
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'  
#unet_mul=copy.deepcopy(unet)
#unet_mul=nn.DataParallel(unet_mul).cuda()

In [161]:
#torch.save(unet_mul,'./unet_example.pth')
#loaded_unet_mul=torch.load('./unet_example.pth')
#loaded_unet_mul

In [165]:
## 多卡：保存&读取模型的权重
#torch.save(unet_mul.state_dict(),'./unet_example.pth')
#loaded_unet_weights_mul=torch,load('./unet_example.pth')
#unet_mul.load_state_dict(loaded_unet_weights_mul)
#unet_mul=nn.DataParallel(unet_mul).cuda()
#unet_mul.state_dict()


#cpu版本不说话

In [ ]:
'''#另外，如果保存的是整个模型，也建议采用提取权重的方式构建的模型：
unet_mul.state_dict = loaded_unet_mul.state_dict
unet_mul =nn.DataParallel(unet_mul).cuda()
unet_mul.state_dict()'''